In [1]:
from tensorflow.keras.models import load_model
from flask import Flask, request, jsonify
import os
import numpy as np
import uuid

# Expected boundaries of the input values
Expected ={
    'cylinders': {'min': 3, 'max': 8}, 
    'displacement': {'min': 68.0, 'max': 455.0}, 
    'horsepower': {'min': 46.0, 'max': 230.0}, 
    'weight': {'min': 1613, 'max': 5140}, 
    'acceleration': {'min': 8.0, 'max': 24.8}, 
    'year': {'min': 70, 'max': 82}, 
    'origin': {'min': 1, 'max': 3}
}

# Setting the directory
os.chdir(r'C:\Users\Wolabtaponikyini\Desktop\Flask Implementation\flaskdata')

# Loading the model
model = load_model(os.path.join(os.getcwd(), 'model.h5'))
#model.summary()

# Creating flask app
app = Flask(__name__)

@app.route('/api', methods=['POST'])
def mpg_prediction():
    content = request.json
    errors = []
    for name in content:
        if name in Expected:
            expected_min = Expected[name]['min']
            expected_max = Expected[name]['max']
            value = content[name]
            if value < expected_min or value > expected_max:
                errors.append(f'Out of bounds: {name}, has value of: {value}, but it should be between {expected_min} and {expected_max}.')
        else:
            errors.append(f'Unexpected field: {name}')
            
    for name in Expected:
        if name not in content:
            errors.append(f'Missing value:  {value}')
    
    if len(errors) < 1:
        x = np.zeros((1,7))
        x[0,0] = content['cylinders']
        x[0,1] = content['displacement']
        x[0,2] = content['horsepower']
        x[0,3] = content['weight']
        x[0,4] = content['acceleration']
        x[0,5] = content['year']
        x[0,6] = content['origin']
        
        prediction = model.predict(x)
        mpg = float(prediction[0])
        response = {'id': str(uuid.uuid4()), 'mpg' : mpg, 'errors': errors}
    else:
        response = {'id': str(uuid.uuid4()), 'errors': errors}
        
    return jsonify(response)



if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)


1/1 [==============================] - 0s 72ms/step


127.0.0.1 - - [21/Jan/2023 23:20:11] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2023 23:42:45] "GET / HTTP/1.1" 404 -


1/1 [==============================] - 0s 23ms/step


127.0.0.1 - - [21/Jan/2023 23:43:11] "POST /api HTTP/1.1" 200 -


In [2]:
#import requests
#json = {
    #'cylinders': 8,
    #'displacement': 300,
    #'horsepower': 78,
    #'weight': 3500,
    #'acceleration': 20,
    #'year': 76,
    #'origin': 1
#}

#r = requests.post('http://localhost:5000/api', json=json)

#if r.status_code == 200:
    #print(f'works: {r.text}')
#else:
    #print(f'Fail: {r.text}')